# Face Tracker using Handle

This ipython file has two Thread classes. One is controlling jetbot's head and motors with handle(joystick) and the other is tracking faces using face detector.

## Remote control thread

In [1]:
from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat
from servoserial import ServoSerial

import os
import cv2
import PID # PID Algorithm
import threading
import time
# Thread function operation library
import inspect
import ctypes

import traitlets
import ipywidgets.widgets as widgets

from IPython.display import display
from uuid import uuid1
from mtcnn.mtcnn import MTCNN

/home/jetbot/Projects/venv/jetbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jetbot/Projects/venv/jetbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jetbot/Projects/venv/jetbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jetbot/Projects/venv/jetbot/lib/python3.6/s

To stop threads..

In [2]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)

def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

create camera initialization instance

In [3]:
controller = widgets.Controller(index=0)  #Replace with the index number of the controller you just tested
display(controller)

Controller()

In [4]:
# camera instance
camera = Camera.instance(width=300, height=300)
image = widgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Open Servo Serial

In [5]:
servo_serial = ServoSerial()

serial Open!


### Define Handle Thread

In [15]:
class Handle_Control(threading.Thread):
    def __init__(self, message, controller, servo, index=0, camera_link=None):
        threading.Thread.__init__(self)
        
        self.message = message
        
#         self.controller = widgets.Controller(index = index)
#         print(self.controller.axes)
#         display(self.controller)

        self.controller = controller
        
        print('Waiting for cotroller...')
        while len(self.controller.axes) == 0:
            time.sleep(1)
        print('Controller connected!')
        
        self.camera_link = camera_link
        
        if self.camera_link != None:
            heartbeat = Heartbeat(period=0.5)
            # Append the callback function to the heartbeat state
            heartbeat.observe(self.handle_heartbeat_status, names='status')
        
        '''Servo Part'''
        if servo == None:
            raise ValueError('servo serial must not be None')
        self.servo_device = servo
        self.updownpulse = 2700
        self.leftrightpulse = 2300
        self.camservoInitFunction()
        
        ''' Robot Part '''
        self.robot = Robot()
        
        
    def handle_heartbeat_status(self, change):
        if change['new'] == Heartbeat.Status.dead:
            self.camera_link.unlink()
            self.robot.stop()
        
    def camUpFunction(self):
        self.updownpulse += 15
        if self.updownpulse > 4095:
            self.updownpulse = 4095
        self.servo_device.Servo_serial_control(2, self.updownpulse)
    
    def camDownFunction(self):
        self.updownpulse -= 15
        if self.updownpulse < 1300:
            self.updownpulse = 1300
        self.servo_device.Servo_serial_control(2, self.updownpulse)
        
    def camLeftFunction(self):
        self.leftrightpulse += 15
        if self.leftrightpulse > 3600:
            self.leftrightpulse = 3600
        self.servo_device.Servo_serial_control(1, self.leftrightpulse)
        
    def camRightFunction(self):
        self.leftrightpulse -= 15
        if self.leftrightpulse < 600:
            self.leftrightpulse = 600
        self.servo_device.Servo_serial_control(1, self.leftrightpulse)
        
    def camservoInitFunction(self):
        self.leftrightpulse = 2300
        self.updownpulse = 2700
        self.servo_device.Servo_serial_control(1, self.leftrightpulse)
        time.sleep(0.1)
        self.servo_device.Servo_serial_control(2, self.updownpulse)
        
    def run(self):
        count1 = count2 = count3 = count4 = count5 = 0
        while 1:
            #Robot car Left and right DC motor
#             if self.controller.axes[1].value <= 0.1:
#                 if (self.controller.axes[0].value <= 0.1 and self.controller.axes[0].value >= -0.1
#                     and self.controller.axes[1].value <= 0.1 and self.controller.axes[1].value >= -0.1):
#                     self.robot.stop()
#                 else:
#                     self.robot.set_motors(-self.controller.axes[1].value + self.controller.axes[0].value, -self.controller.axes[1] - self.controller.axes[0].value)

#                 time.sleep(0.01)
#             else:
#                 self.robot.set_motors(-self.controller.axes[1].value - self.controller.axes[0].values, -self.controller.axes[1].value + self.controller.axes[0].value)
#                 time.sleep(0.01)

            if controller.axes[1].value <= 0:
                self.robot.set_motors(-self.controller.axes[1].value + self.controller.axes[0].value, -self.controller.axes[1].value - self.controller.axes[0].value)
                time.sleep(0.01)
            else:
                self.robot.set_motors(-self.controller.axes[1].value - self.controller.axes[0].value, -self.controller.axes[1].value + self.controller.axes[0].value)
                time.sleep(0.01)
                
            # Servo control cameara up and down
            if self.controller.axes[5].value >= 0.5:
                count1 += 1
                if count1 >= 3:
                    self.camDownFunction()
                    count1 = 0
            elif self.controller.axes[5].value <= -0.5:
                count1 += 1
                if count1 >= 3:
                    self.camUpFunction()
                    count1 = 0
            else:
                count1 = 0
                
            # Servo control camera left and right
            if self.controller.axes[2].value <= -0.5:
                count2 += 1
                if count2 >= 3:
                    self.camRightFunction()
                    count2 = 0
            elif self.controller.axes[2].value >= 0.5:
                count2 += 1
                if count2 >= 3:
                    self.camLeftFunction()
                    count2 = 0
            else:
                count2 = 0
                
            # Servo control camera up down, left and right is reset
            if self.controller.buttons[8].value == True:
                count3 += 1
                if count3 >= 3:
                    self.camservoInitFunction()
                    count3 = 0
            else:
                count3 = 0
                
            # Servo control servo rise and decline
            if self.controller.buttons[6].value == True:
                count4 += 1
                if count4 >= 3:
                    self.robot.set_vertical_motors(1)
                    count4 = 0
            elif self.controller.buttons[4].value == True:
                count4 += 1
                if count4 >= 3:
                    self.robot.set_vertical_motors(-1)
                    count4 =0
            else:
                self.robot.set_vertical_motors(0)
                count4 = 0

In [18]:
thread2 = Handle_Control(message='Handle', controller=controller, servo=servo_serial, index=0, camera_link=camera_link)
thread2.setDaemon(True)
thread2.start()

Waiting for cotroller...
Controller connected!
188
b'\xff\xff\x01\x07\x03*\x08\xfc\x00\n\xbc'
41
b'\xff\xff\x02\x07\x03*\n\x8c\x00\n)'


In [19]:
stop_thread(thread2)

## Face Tracker Thread

### Define detector

In [6]:
detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [13]:
class Face_Tracker(threading.Thread):
    def __init__(self, message, camera, servo, detector, image=None):
        threading.Thread.__init__(self)
        
        self.message = message
        
        ''' camera part '''
        if camera == None:
            raise ValueError("camera instance can\'t be None")
        self.camera = camera
        
        ''' servo part '''
        if servo == None:
            raise ValueError('servo serial must not be None')
        self.servo_device = servo
        self.target_valuex = 2300
        self.target_valuey = 2700
        
        ''' face detector '''
        self.detector = detector
        
        self.image_widget = image
        
        ''' PID control instance '''
        self.xservo_pid = PID.PositionalPID(1.9, 0.4, 0.35)
        self.yservo_pid = PID.PositionalPID(1.5, 0.3, 0.3)
        
        
        self.lock = threading.Lock()
        
    def run(self):
        while 1:
            frame = self.camera.value
            img = frame.copy()
            faces = self.detector.detect_faces(img)
            # if there's faces in the frame
            if len(faces) > 0:
                (face_x, face_y, face_w, face_h) = faces[0]['box'] # get coordinates
                # draw rectangle
                cv2.rectangle(frame, (face_x, face_y), (face_x+face_w, face_y+face_h), color=(0,255,0), thickness=2)
                
                # follow face based on PID algorithm
                self.xservo_pid.SystemOutput = face_x + face_w/2
                self.xservo_pid.SetStepSignal(150)
                self.xservo_pid.SetInertiaTime(0.01, 0.006)
                self.target_valuex = int(2300 + self.xservo_pid.SystemOutput*3)
                
                self.yservo_pid.SystemOutput = face_y + face_h/2
                self.yservo_pid.SetStepSignal(150)
                self.yservo_pid.SetInertiaTime(0.01, 0.006)
                self.target_valuey = int(2700 + self.yservo_pid.SystemOutput*3)
                
                self.servo_device.Servo_serial_double_control(1, self.target_valuex, 2, self.target_valuey)
                
            self.lock.acquire()
            if self.image_widget:
                self.image_widget.value = bgr8_to_jpeg(frame)
            self.lock.release()

In [14]:
thread_face = Face_Tracker('face', camera, servo_serial, detector, None)
thread_face.setDaemon(True)
thread_face.start()

140
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x05\x00\n\x02\n\x88\x00\n\x8c'
1
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\x91\x00\n\x02\n\x88\x00\n\x01'
173
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x16\x00\n\x02\x0bT\x00\n\xad'
90
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\xea\x00\n\x02\x0b\xd3\x00\nZ'
8
b"\xff\xff\xfe\x0e\x83*\x04\x01\x08\xe9\x00\n\x02\x0b'\x00\n\x08"
211
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08h\x00\n\x02\n\xde\x00\n\xd3'
204
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xef\x00\n\x02\n^\x00\n\xcc'
238
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xc8\x00\n\x02\x0bb\x00\n\xee'
109
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xbb\x00\n\x02\x0b\xf0\x00\nm'
192
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x1c\x00\n\x02\x0b;\x00\n\xc0'
47
b'\xff\xff\xfe\x0e\x83*\x04\x01\ty\x00\n\x02\x0bo\x00\n/'
177
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\xe4\x00\n\x02\x0b\x82\x00\n\xb1'
134
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x1f\x00\n\x02\x0bq\x00\n\x86'
146
b'\xff\xff\xfe\x0e\x83*\x04\x01\n$\x00\n\x02\x0b`\x00\n\x92'
243
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\xc7\x00\n\

155
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x0e\x00\n\x02\x0bn\x00\n\x9b'
106
b'\xff\xff\xfe\x0e\x83*\x04\x01\tE\x00\n\x02\x0bh\x00\nj'
60
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x84\x00\n\x02\x0bW\x00\n<'
29
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\xb1\x00\n\x02\x0bI\x00\n\x1d'
14
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xf0\x00\n\x02\x0b\x1a\x00\n\x0e'
167
b'\xff\xff\xfe\x0e\x83*\x04\x01\t@\x00\n\x02\x0b0\x00\n\xa7'
54
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x83\x00\n\x02\x0b^\x00\n6'
225
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\xba\x00\n\x02\x0b|\x00\n\xe1'
27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xbb\x00\n\x02\x0bB\x00\n\x1b'
5
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x01\x00\n\x02\x0b\x12\x00\n\x05'
34
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xb9\x00\n\x02\x0b=\x00\n"'
181
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x10\x00\n\x02\x0bR\x00\n\xb5'
218
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xf7\x00\n\x02\x0bG\x00\n\xda'
33
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xc0\x00\n\x02\x0b7\x00\n!'
63
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xa5\x00\n\x02\x0b4\x0

129
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\x07\x00\n\x02\x0b\x8d\x00\n\x81'
129
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\x06\x00\n\x02\x0b\x8e\x00\n\x81'
249
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\xb4\x00\n\x02\x0bh\x00\n\xf9'
172
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0c\x1d\x00\n\x02\x0bK\x00\n\xac'
86
b"\xff\xff\xfe\x0e\x83*\x04\x01\x0b'\x00\n\x02\x0b\x98\x00\nV"
180
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xa1\x00\n\x02\x0b\xc1\x00\n\xb4'
101
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b%\x00\n\x02\x0b\x8b\x00\ne'
69
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b]\x00\n\x02\x0bs\x00\nE'
84
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0bH\x00\n\x02\x0by\x00\nT'
108
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\x1b\x00\n\x02\x0b\x8e\x00\nl'
114
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\x04\x00\n\x02\x0b\x9f\x00\nr'
117
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\x05\x00\n\x02\x0b\x9b\x00\nu'
115
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\x12\x00\n\x02\x0b\x90\x00\ns'
116
b'\xff\xff\xfe\x0e\x83*\x04\x01\x0b\x18\x00\n\x02\x0b\x89\x00\nt'
116
b'\xff\xff\xfe\x0e\x83*\x04\x0

251
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xbf\x00\n\x02\x0b\\\x00\n\xfb'
13
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xbc\x00\n\x02\x0bM\x00\n\r'
17
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xbb\x00\n\x02\x0bJ\x00\n\x11'
16
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xba\x00\n\x02\x0bL\x00\n\x10'
22
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xb5\x00\n\x02\x0bK\x00\n\x16'
33
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xad\x00\n\x02\x0bH\x00\n!'
49
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xa1\x00\n\x02\x0bD\x00\n1'
71
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x8f\x00\n\x02\x0b@\x00\nG'
97
b'\xff\xff\xfe\x0e\x83*\x04\x01\nx\x00\n\x02\x0b=\x00\na'
125
b'\xff\xff\xfe\x0e\x83*\x04\x01\n[\x00\n\x02\x0b>\x00\n}'
160
b'\xff\xff\xfe\x0e\x83*\x04\x01\n8\x00\n\x02\x0b>\x00\n\xa0'
193
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x18\x00\n\x02\x0b=\x00\n\xc1'
214
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x03\x00\n\x02\x0b=\x00\n\xd6'
26
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\xbc\x00\n\x02\x0bA\x00\n\x1a'
82
b'\xff\xff\xfe\x0e\x83*\x04\x01\t~\x00\n\x02\x0bG\x00\nR'
101
b'\xff\xff\xfe\x0

111
b'\xff\xff\xfe\x0e\x83*\x04\x01\nU\x00\n\x02\x0bR\x00\no'
89
b'\xff\xff\xfe\x0e\x83*\x04\x01\nk\x00\n\x02\x0bR\x00\nY'
64
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x85\x00\n\x02\x0bQ\x00\n@'
56
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x93\x00\n\x02\x0bK\x00\n8'
45
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xa2\x00\n\x02\x0bG\x00\n-'
40
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xaa\x00\n\x02\x0bD\x00\n('
44
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\xa5\x00\n\x02\x0bE\x00\n,'
56
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x96\x00\n\x02\x0bH\x00\n8'
70
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x84\x00\n\x02\x0bL\x00\nF'
73
b'\xff\xff\xfe\x0e\x83*\x04\x01\n|\x00\n\x02\x0bQ\x00\nI'
69
b'\xff\xff\xfe\x0e\x83*\x04\x01\n|\x00\n\x02\x0bU\x00\nE'
64
b'\xff\xff\xfe\x0e\x83*\x04\x01\n~\x00\n\x02\x0bX\x00\n@'
56
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x83\x00\n\x02\x0b[\x00\n8'
47
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x88\x00\n\x02\x0b_\x00\n/'
41
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x8e\x00\n\x02\x0b_\x00\n)'
34
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x95\x00\n\x02\x0b

135
b'\xff\xff\xfe\x0e\x83*\x04\x01\x04\xe4\x00\n\x02\x0c\xb0\x00\n\x87'
182
b'\xff\xff\xfe\x0e\x83*\x04\x01\x04\xa3\x00\n\x02\x0c\xc2\x00\n\xb6'
33
b'\xff\xff\xfe\x0e\x83*\x04\x01\x04/\x00\n\x02\x0c\xcb\x00\n!'
250
b'\xff\xff\xfe\x0e\x83*\x04\x01\x03\x95\x00\n\x02\x0c\x8d\x00\n\xfa'
193
b'\xff\xff\xfe\x0e\x83*\x04\x01\x02\xfa\x00\n\x02\x0cb\x00\n\xc1'
54
b'\xff\xff\xfe\x0e\x83*\x04\x01\x02\x9a\x00\n\x02\x0cM\x00\n6'
127
b'\xff\xff\xfe\x0e\x83*\x04\x01\x02h\x00\n\x02\x0c6\x00\n\x7f'
141
b'\xff\xff\xfe\x0e\x83*\x04\x01\x02X\x00\n\x02\x0c8\x00\n\x8d'
144
b'\xff\xff\xfe\x0e\x83*\x04\x01\x02X\x00\n\x02\x0c5\x00\n\x90'
173
b'\xff\xff\xfe\x0e\x83*\x04\x01\x02X\x00\n\x02\x0c\x18\x00\n\xad'
241
b'\xff\xff\xfe\x0e\x83*\x04\x01\x03R\x00\n\x02\x0b\xda\x00\n\xf1'
122
b'\xff\xff\xfe\x0e\x83*\x04\x01\x03\xa0\x00\n\x02\x0c\x02\x00\nz'
250
b'\xff\xff\xfe\x0e\x83*\x04\x01\x03\x9e\x00\n\x02\x0c\x84\x00\n\xfa'
89
b'\xff\xff\xfe\x0e\x83*\x04\x01\x04\x1a\x00\n\x02\x0c\xa8\x00\nY'
252
b'\xff\xff\xfe\x0e\x83

In [12]:
stop_thread(thread_face)

In [23]:
del camera

In [11]:
servo_serial.Servo_serial_double_control(1, 2700, 2, 2700)

255
b'\xff\xff\xfe\x0e\x83*\x04\x01\n\x8c\x00\n\x02\n\x8c\x00\n\xff'
40
b'\xff\xff\xfe\x0e\x83*\x04\x01\t9\x00\n\x02\n\xb7\x00\n('
41
b'\xff\xff\xfe\x0e\x83*\x04\x01\t~\x00\n\x02\nq\x00\n)'
41
b'\xff\xff\xfe\x0e\x83*\x04\x01\nG\x00\n\x02\n\xa7\x00\n)'
38
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\xf7\x00\n\x02\n\xfb\x00\n&'
129
b'\xff\xff\xfe\x0e\x83*\x04\x01\tV\x00\n\x02\nA\x00\n\x81'
142
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xcf\x00\n\x02\n\xbc\x00\n\x8e'
161
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08n\x00\n\x02\x0b\t\x00\n\xa1'
233
b'\xff\xff\xfe\x0e\x83*\x04\x01\x07\xeb\x00\n\x02\x0bE\x00\n\xe9'
63
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xae\x00\n\x02\x0b+\x00\n?'
235
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x00\x00\n\x02\x0b,\x00\n\xeb'
14
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xd6\x00\n\x02\x0b4\x00\n\x0e'
14
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xcc\x00\n\x02\x0b>\x00\n\x0e'
244
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xdc\x00\n\x02\x0bH\x00\n\xf4'
131
b'\xff\xff\xfe\x0e\x83*\x04\x01\tA\x00\n\x02\x0bS\x00\n\x83'
48
b